In [ ]:
#====================================================================================================#
#                                                                                                    #
#                                                        ██╗   ██╗   ████████╗ █████╗ ██████╗        #
#      Competición - INAR AEC2                           ██║   ██║   ╚══██╔══╝██╔══██╗██╔══██╗       #
#                                                        ██║   ██║█████╗██║   ███████║██║  ██║       #
#      created:        05/12/2025  -  11:20:05           ██║   ██║╚════╝██║   ██╔══██║██║  ██║       #
#      last change:    05/12/2025  -  14:55:18           ╚██████╔╝      ██║   ██║  ██║██████╔╝       #
#                                                         ╚═════╝       ╚═╝   ╚═╝  ╚═╝╚═════╝        #
#                                                                                                    #
#      Ismael Hernandez Clemente                         ismael.hernandez@live.u-tad.com             #
#                                                                                                    #
#      Github:                                           https://github.com/ismaelucky342            #
#                                                                                                    #
#====================================================================================================#

# Iteración 3: (Regularización)
**Estado: DEPRECATED**

En este experimento, he intentado aplicar una regularización extrema (L2 más alta, Dropout del 0.7) para ver si podía eliminar por completo el sobreajuste. Aunque el modelo se volvió muy estable, el rendimiento global no superó a la V2, lo que sugiere que estábamos empezando a sufrir de *underfitting* en algunas áreas.

In [ ]:
# Configuro variables de entorno y verifico protobuf.
import os
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
import sys
try:
    import google.protobuf
    if hasattr(google.protobuf, '__version__'):
except:
    pass

In [ ]:
# Importo las bibliotecas con TensorFlow y regularizadores.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
seed = 42
np.random.seed(seed)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Embedding, LSTM, Bidirectional, Dense, Dropout, 
    GlobalMaxPooling1D, Conv1D, SpatialDropout1D
)
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
tf.random.set_seed(seed)
keras.utils.set_random_seed(seed)
pd.set_option('display.max_rows', 36)
pd.set_option("display.max_colwidth", 150)

In [ ]:
# Defino hiperparámetros.
MAX_WORDS = 10000
MAX_LEN = 200
EMBEDDING_DIM = 100
LSTM_UNITS = 64           
DENSE_UNITS = 32          
DROPOUT_RATE = 0.7        
SPATIAL_DROPOUT = 0.4     
L2_REG = 5e-4             
BATCH_SIZE = 32
EPOCHS = 50
VALIDATION_SPLIT = 0.2
LEARNING_RATE = 5e-4      
CLIPNORM = 1.0            
estimated_params_v3 = (
    MAX_WORDS * EMBEDDING_DIM +
    4 * LSTM_UNITS * (EMBEDDING_DIM + LSTM_UNITS + 1) * 2 +
    (LSTM_UNITS * 2) * DENSE_UNITS + DENSE_UNITS +
    DENSE_UNITS + 1
)

In [ ]:
# Cargo los datos de entrenamiento.
train = pd.read_csv("/kaggle/input/u-tad-spam-not-spam-2025-edition/train.csv", index_col="row_id")
train.head(10)

In [ ]:
# Analizo la longitud de los textos con gráficos simplificados.
train['text_length'] = train['text'].apply(lambda x: len(str(x).split()))
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.hist(train['text_length'], bins=50, edgecolor='black', alpha=0.7)
plt.title('Distribución de Longitud de Textos')
plt.xlabel('Número de palabras')
plt.ylabel('Frecuencia')
plt.axvline(train['text_length'].mean(), color='red', linestyle='--', label=f'Media: {train["text_length"].mean():.1f}')
plt.legend()
plt.grid(True, alpha=0.3)
plt.subplot(1, 2, 2)
sns.boxplot(data=train, x='spam_label', y='text_length')
plt.title('Longitud por Clase')
plt.xlabel('Clase (0=Not SPAM, 1=SPAM)')
plt.ylabel('Número de palabras')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Tokenizo los textos y preparo train/val.
X_train_text = train['text'].values
y_train = train['spam_label'].values
tokenizer = Tokenizer(num_words=MAX_WORDS, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train_text)
X_train_seq = tokenizer.texts_to_sequences(X_train_text)
X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding='post', truncating='post')
X_train_final, X_val, y_train_final, y_val = train_test_split(
    X_train_pad, y_train, 
    test_size=VALIDATION_SPLIT, 
    random_state=seed,
    stratify=y_train
)

In [ ]:
# Construyo el modelo 
def build_model_v3():
    model = Sequential([
        Embedding(
            input_dim=MAX_WORDS,
            output_dim=EMBEDDING_DIM,
            input_length=MAX_LEN,
            name='embedding'
        ),
        SpatialDropout1D(SPATIAL_DROPOUT),
        Bidirectional(
            LSTM(
                LSTM_UNITS, 
                return_sequences=True,
                kernel_regularizer=l2(L2_REG),
                recurrent_regularizer=l2(L2_REG),
                bias_regularizer=l2(L2_REG)     
            ), 
            name='bidirectional_lstm'
        ),
        GlobalMaxPooling1D(),
        Dense(
            DENSE_UNITS, 
            activation='relu',
            kernel_regularizer=l2(L2_REG),
            bias_regularizer=l2(L2_REG),        
            name='dense_1'
        ),
        Dropout(DROPOUT_RATE),
        Dense(1, activation='sigmoid', name='output')
    ], name='spam_classifier_v3_shock_therapy')
    return model
model = build_model_v3()
optimizer = keras.optimizers.AdamW(
    learning_rate=LEARNING_RATE,
    weight_decay=1e-4,
    clipnorm=CLIPNORM  
)
model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=[
        'accuracy',
        keras.metrics.Precision(name='precision'),
        keras.metrics.Recall(name='recall'),
        keras.metrics.AUC(name='auc')
    ]
)
model.build(input_shape=(None, MAX_LEN))
model.summary()

In [ ]:
# Entreno con paciencia mínima 
callbacks = [
    EarlyStopping(
        monitor='val_loss',
        patience=2,  
        restore_best_weights=True,
        verbose=1
    ),
    ModelCheckpoint(
        'best_spam_model_v3.keras',
        monitor='val_loss',
        save_best_only=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=1,  
        min_lr=1e-6,
        verbose=1
    )
]
history = model.fit(
    X_train_final, y_train_final,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(X_val, y_val),
    callbacks=callbacks,
    verbose=1
)

In [ ]:
# Predigo en validación y genero la matriz de confusión V3.
y_pred_proba = model.predict(X_val)
y_pred = (y_pred_proba > 0.5).astype(int).flatten()
mcc_score = matthews_corrcoef(y_val, y_pred)
plt.figure(figsize=(8, 6))
cm = confusion_matrix(y_val, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Reds', 
            xticklabels=['Not SPAM', 'SPAM'],
            yticklabels=['Not SPAM', 'SPAM'])
plt.title(f'Confusion Matrix V3 - Shock Therapy (MCC: {mcc_score:.4f})')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

In [ ]:
# Genero las curvas de aprendizaje para analizar el efecto de la regularización.
def plot_learning_curves(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs_range = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Training Accuracy')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.legend()
    plt.grid(True, alpha=0.3)

    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

plot_learning_curves(history)

In [ ]:
# Cargo los datos de prueba.
test = pd.read_csv("/kaggle/input/u-tad-spam-not-spam-2025-edition/test.csv", index_col="row_id")
test.head()

In [ ]:
# Tokenizo y predigo en el conjunto de prueba.
X_test_text = test['text'].values
X_test_seq = tokenizer.texts_to_sequences(X_test_text)
X_test_pad = pad_sequences(X_test_seq, maxlen=MAX_LEN, padding='post', truncating='post')
y_pred_proba_test = model.predict(X_test_pad, batch_size=BATCH_SIZE)
y_pred_test = (y_pred_proba_test > 0.5).astype(int).flatten()

In [ ]:
# Creo el archivo de submission para V3.
submission = pd.read_csv("/kaggle/input/u-tad-spam-not-spam-2025-edition/sample_submission.csv")
submission["spam_label"] = y_pred_test
submission.to_csv('submission.csv', index=False)
submission.head(10)

## Análisis de la Iteración 3
Este experimento de de regularizar un poco a lo bestia ha demostrado que existe un límite para la regularización. Aunque el sobreajuste es casi inexistente, el MCC ha bajado ligeramente respecto a la V2. El modelo es demasiado "rígido" para capturar la complejidad del lenguaje en algunos mensajes de spam.

**Conclusión:** Volveré a los niveles de regularización de la V2 pero cambiaré la arquitectura para mejorar la extracción de características.